In [73]:
import sys
sys.path.insert(0, r'C:\Users\HARSH\Documents\chatweb3-backend')
from utils import *
set_api_key()

In [74]:
import re
import requests

from langchain.llms import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.prompts.base import BaseOutputParser


TEMPLATE = \
'''
Query: {query}
Note: The query is related to the blockchain ecosystem.
Give 10 distinct related searches most asked on the web.
1.'''

class ChatOutputParser(BaseOutputParser):

    @property
    def _type(self) -> str:
        """Return the type key."""
        return "chat_output_parser"

    def parse(self, text: str) -> str:
        """Parse the output of an LLM call."""
        text = text.strip()
        text = re.sub(r"\d+. ", '', text)
        text = text.split('\n')
        return text

In [75]:

class SearchURLs():
    def __init__(self,) -> None:
        super().__init__()
        llm = OpenAI(
            temperature=0.6, max_tokens=-1,
        )
        output_parser = ChatOutputParser()
        relSearch_prompt = PromptTemplate(
            input_variables=["query"],
            template=TEMPLATE,
            output_parser=output_parser,
        )
        self.chain = LLMChain(llm=llm, prompt=relSearch_prompt, verbose=True)

        subscription_key="df01e65a4da8434b841e8a81b34c9d25"
        self.search_url = "https://api.bing.microsoft.com/v7.0/search"
        self.headers = {"Ocp-Apim-Subscription-Key" : subscription_key}

    def via_llm(self, query): 
        relSearches = self.chain.apply_and_parse([{"query": query}])[0]
        return relSearches

    def via_bing(self, query):
        params  = {"q": query, "textDecorations": True, "textFormat": "HTML"}
        response = requests.get(self.search_url, headers=self.headers, params=params)
        response.raise_for_status()
        response = response.json()
        relSearches = [r['text'] for r in response['relatedSearches']['value']]
        return relSearches

In [76]:
s = SearchURLs()

In [77]:
s.via_llm("What is a 51% attack?")

Prompt after formatting:

Query: What is a 51% attack?
Note: The query is related to the blockchain ecosystem.
Give 10 distinct related searches most asked on the web.
1.


['What is a attack in cryptocurrency?',
 'What is a double spend attack?',
 'How does a attack work?',
 'What is a majority attack?',
 'What is a attack on Bitcoin?',
 'How to prevent a attack?',
 'What is the cost of a attack?',
 'What is a selfish mining attack?',
 'How to detect a attack?',
 'What is a attack on Ethereum?']

In [78]:
s.via_bing("What is a 51% attack?")

['51% attack blockchain',
 '51% attack bitcoin',
 '51 percent attack',
 'attacks on blockchain',
 'what is 51% attack',
 'proof of work 51% attack',
 'bitcoin 51% attack cost',
 '51% attack example']